In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset("C:\\Users\\norma\\.cate\\data_stores\\esa_cci_odp\\esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1\\20030819120000-ESACCI-L4_GHRSST-SSTdepth-OSTIA-GLOB_LT-v02.0-fv01.1.nc")

In [3]:
ds

<xarray.Dataset>
Dimensions:                 (bnds: 2, lat: 3600, lon: 7200, time: 1)
Coordinates:
  * time                    (time) datetime64[ns] 2003-08-19T12:00:00
  * lat                     (lat) float32 -89.975 -89.925 -89.875 -89.825 ...
  * lon                     (lon) float32 -179.975 -179.925 -179.875 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds               (time, bnds) datetime64[ns] 2003-08-19 2003-08-20
    lat_bnds                (lat, bnds) float32 -90.0 -89.95 -89.95 -89.9 ...
    lon_bnds                (lon, bnds) float32 -180.0 -179.95 -179.95 ...
    analysed_sst            (time, lat, lon) float64 nan nan nan nan nan nan ...
    analysis_error          (time, lat, lon) float64 nan nan nan nan nan nan ...
    sea_ice_fraction        (time, lat, lon) float64 nan nan nan nan nan nan ...
    sea_ice_fraction_error  (time, lat, lon) float64 nan nan nan nan nan nan ...
    mask                    (time, lat, lon) float64 2.0 2.0 2.0 2.0 2.0

In [13]:
ds.dims

Frozen(SortedKeysDict({'time': 1, 'lat': 3600, 'lon': 7200, 'bnds': 2}))

In [5]:
ds.coords

Coordinates:
  * time     (time) datetime64[ns] 2003-08-19T12:00:00
  * lat      (lat) float32 -89.975 -89.925 -89.875 -89.825 -89.775 -89.725 ...
  * lon      (lon) float32 -179.975 -179.925 -179.875 -179.825 -179.775 ...

In [18]:
var = ds['analysed_sst']
var

<xarray.DataArray 'analysed_sst' (time: 1, lat: 3600, lon: 7200)>
[25920000 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2003-08-19T12:00:00
  * lat      (lat) float32 -89.975 -89.925 -89.875 -89.825 -89.775 -89.725 ...
  * lon      (lon) float32 -179.975 -179.925 -179.875 -179.825 -179.775 ...
Attributes:
    units:          kelvin
    long_name:      analysed sea surface temperature
    valid_min:      -300
    valid_max:      4500
    standard_name:  sea_water_temperature
    depth:          20 cm
    source:         ATSR<1,2>-ESACCI-L3U-v1.0, AATSR-ESACCI-L3U-v1.0, AVHRR<1...
    comment:        SST analysis produced for ESA SST CCI project using the O...

In [26]:
var.shape

(1, 3600, 7200)

In [27]:
var.dims

('time', 'lat', 'lon')

In [20]:
var.coords

Coordinates:
  * time     (time) datetime64[ns] 2003-08-19T12:00:00
  * lat      (lat) float32 -89.975 -89.925 -89.875 -89.825 -89.775 -89.725 ...
  * lon      (lon) float32 -179.975 -179.925 -179.875 -179.825 -179.775 ...

In [22]:
'lon' in var.coords

True

In [24]:
var.coords['lon']

<xarray.DataArray 'lon' (lon: 7200)>
array([-179.975006, -179.925003, -179.875   , ...,  179.875   ,  179.925003,
        179.975006], dtype=float32)
Coordinates:
  * lon      (lon) float32 -179.975 -179.925 -179.875 -179.825 -179.775 ...
Attributes:
    standard_name:    longitude
    units:            degrees_east
    valid_min:        -180.0
    valid_max:        180.0
    axis:             X
    reference_datum:  geographical coordinates, WGS84 projection
    bounds:           lon_bnds
    comment:          
    long_name:        Longitude

In [29]:
{k: len(v) for k, v in var.coords.items()}

{'lat': 3600, 'lon': 7200, 'time': 1}

### Given

* `ds`: an xarray dataset
* `res_name`: name of *resource*
* `var_name`: name of *variable* in *resource*
* `var_index`: an array whose length is equal to the number of dimensions
* `by_label`: bool saying if indexing is by label or by integer

In [58]:
def get_effective_index(var, indexes):
    num_dims = len(var.dims)
    effective_index = dict()
    for i in range(num_dims):
        dim_name = var.dims[i]
        dim_size = var.shape[i]
        index = indexes[i]
        if index is None:
            if dim_size == 1:
                effective_index[dim_name] = 0
            else:
                effective_index[dim_name] = slice(None, None)
        else:
            effective_index[dim_name] = index
    
    return effective_index

In [59]:
def get_subset(ds, var_name, indexes, by_label):
    var = ds[var_name]
    effective_index = get_effective_index(var, indexes)
    return var.loc[effective_index] if by_label else var[effective_index]

In [61]:
# We get from server
var_name = 'analysed_sst'
indexes = [None, 0.0, None]

print(get_effective_index(ds[var_name], indexes))

subset = get_subset(ds, var_name, indexes, True)
subset

{'time': 0, 'lat': 0.0, 'lon': slice(None, None, None)}


KeyError: 0

In [41]:
subset = ds['analysed_sst'].sel(method='nearest', **effective_index)
subset

<xarray.DataArray 'analysed_sst' (lon: 7200)>
array([ 302.199993,  302.189993,  302.179993, ...,  302.239993,  302.229993,
        302.219993])
Coordinates:
    time     datetime64[ns] 2003-08-19T12:00:00
    lat      float32 0.025
  * lon      (lon) float32 -179.975 -179.925 -179.875 -179.825 -179.775 ...
Attributes:
    units:          kelvin
    long_name:      analysed sea surface temperature
    valid_min:      -300
    valid_max:      4500
    standard_name:  sea_water_temperature
    depth:          20 cm
    source:         ATSR<1,2>-ESACCI-L3U-v1.0, AATSR-ESACCI-L3U-v1.0, AVHRR<1...
    comment:        SST analysis produced for ESA SST CCI project using the O...

In [53]:
p = subset.to_dataframe()
p.drop('lon')

ValueError: labels ['lon'] not contained in axis

In [51]:
p.to_csv()

'lon,time,lat,analysed_sst\n-179.97500610351562,2003-08-19 12:00:00,0.02500000037252903,302.1999932471663\n-179.9250030517578,2003-08-19 12:00:00,0.02500000037252903,302.1899932473898\n-179.875,2003-08-19 12:00:00,0.02500000037252903,302.1799932476133\n-179.8249969482422,2003-08-19 12:00:00,0.02500000037252903,302.15999324806035\n-179.77499389648438,2003-08-19 12:00:00,0.02500000037252903,302.1399932485074\n-179.72500610351562,2003-08-19 12:00:00,0.02500000037252903,302.1299932487309\n-179.6750030517578,2003-08-19 12:00:00,0.02500000037252903,302.10999324917793\n-179.625,2003-08-19 12:00:00,0.02500000037252903,302.08999324962497\n-179.5749969482422,2003-08-19 12:00:00,0.02500000037252903,302.069993250072\n-179.52499389648438,2003-08-19 12:00:00,0.02500000037252903,302.069993250072\n-179.47500610351562,2003-08-19 12:00:00,0.02500000037252903,302.04999325051904\n-179.4250030517578,2003-08-19 12:00:00,0.02500000037252903,302.03999325074255\n-179.375,2003-08-19 12:00:00,0.02500000037252903